In [1]:
import json
import pandas as pd

# data clean
import re # punctuation,data clean

# tokenization and feature extraction
# install expacted word bag for jieba
import jieba 
jieba.load_userdict('jieba_expanded_dict.txt')
import jieba.analyse # TF-IDF 
import numpy as np # TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF

# models
from sklearn.ensemble import RandomForestClassifier # RF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC # SVM
from sklearn.neural_network import MLPClassifier # MLP
from sklearn.model_selection import RandomizedSearchCV # MLP


# analyse
import seaborn as sns # density curve 
import matplotlib.pyplot as plt # pie chart

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.755 seconds.
Prefix dict has been built successfully.


In [2]:
balanced_df_all_cleaned_tokenization = pd.read_csv('balanced_df_all_cleaned_tokenization.csv')

In [3]:
balanced_df_all_cleaned_tokenization2 = balanced_df_all_cleaned_tokenization.copy()

In [14]:
balanced_df_all_cleaned_tokenization2

,ID,label,tweet_content
0,12818,0,机 掉落 样 营养成分 表审 姥爷 手机 里 发现 两张 片 喜欢 号弃 博久 定时 出没 ...
1,15046,0,平安 健康 乐观 开心 快乐 关注 力量 发声 入 携手 久 没见 日常 外国 居住 条例 ...
2,8860,0,翻转 开运 卡 红包 刚刚 获送 金 红包 集齐 开运 卡夕 瓜分 亿 红包 玩 戳 新年 ...
3,3523,0,蒽 确实 确实 确实 确实 确实 确实 终 说句话 确实 收回 夸夸 工作 太 白天 吐槽 ...
4,15252,0,永远 没 发生 事情 担心 握美 明天 叶 紫檀 手串 暖金盘 里 点 酥 山拟 君王 子细...
...,...,...,...
20645,24223,1,思 诺思 舒乐安定 代开 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合...
20646,29630,1,讨厌 公共交通 海底 低谷 期熬 温柔 次 教训 快乐 早 没 没 唯独 快乐 想 瞬间 快...
20647,30838,1,熬 月夜 周 天天 早 胸闷 气短 难真 熬 吃药 旧 失眠 效 时间 越 越短 候觉 精神...
20648,28498,1,陕西 黑布 林 天然 优质 产 精选 果 肉质 酸甜 汁 味道鲜美 果型 丰满 农药 催熟 ...


In [43]:
# 取50行label为0，50行label为1
df_label_0 = balanced_df_all_cleaned_tokenization2[balanced_df_all_cleaned_tokenization2['label'] == 0].sample(n=50, random_state=42)
df_label_1 = balanced_df_all_cleaned_tokenization2[balanced_df_all_cleaned_tokenization2['label'] == 1].sample(n=50, random_state=42)

# 合并两个dataframe
balanced_df_all_cleaned_tokenization2 = pd.concat([df_label_0, df_label_1])

print(balanced_df_all_cleaned_tokenization2)

          ID  label                                      tweet_content
10148  14076      0  燃强 兵夜 集结 号响 易队 运筹帷幄 决胜 街舞 舞台 易燃 装置 蓄势 发 拭目 手指 ...
1692    8412      0  市 持续 雨雪 冰冻 天气 交通 出行 情况 备受 关注 日 午时 许记 怀化市 交通运输 ...
2692    2749      0  王国 韩国 古代 版 丧尸 篇季 宣传 季节 奏 紧凑 容错 画面 精彩 期季 唐顿 庄园 ...
7703    3883      0  穿 秋衣 秋裤 日 辈子 父母 条件 付出 正馆 快手 视频 分享 视频 日 阳光 开朗 乐...
321    20308      0  前乐视 航天 金税 老说 工商 税务 税务 厅 卖 税控 盘 开票 税盘 厂家 说 税务 关...
...      ...    ...                                                ...
13388  27339      1  日岁 女子 岛美奈 年前 诊断 出患重 系统 退化 症 病原 明 没效 治疗 方法 终 身体...
17587  30332      1  隐私 没错 游戏 憨批 蠢批 毛病 真 傻逼 焦灼 早睡 早 六盘水 妈 医院 医生 真 脾...
13695  23484      1  关注 组合 超话 伙伴 快 组合 超话 聊聊 戳 里 组合 关注 超话 伙伴 快 超话 聊聊...
13812  28741      1  恶心 家然 神病觉 难受 投入 时间 精力 没想 东西 瞬间 失落 灌满 样子 感值 确实 ...
10636  27569      1  五 快乐 橙子 劈叉 心 西装 弹 拍图 成子 太爱 成子 拍 翻旧 点点 找 细节 玩 没...

[100 rows x 3 columns]


In [15]:
# replace NaN values with an empty string
balanced_df_all_cleaned_tokenization2['tweet_content'] = balanced_df_all_cleaned_tokenization2['tweet_content'].fillna('')

In [16]:
# create a TfidfVectorizer object 将tweet_content列作为特征向量化
vectorizer = TfidfVectorizer()

In [17]:
# fit and transform the data
X = vectorizer.fit_transform(balanced_df_all_cleaned_tokenization2['tweet_content'])

In [18]:
# 将label列作为目标变量
y = balanced_df_all_cleaned_tokenization2['label']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [20]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV

In [36]:
# test parameters

In [50]:
# 定义超参数空间
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'max_iter': [1000, 1500, 2000]
}

In [51]:
# 定义分类器
mlp = MLPClassifier()
# 使用RandomizedSearchCV搜索最佳超参数组合
# GridSearchCV和RandomizedSearchCV都是用于超参数调优的方法，但它们的实现方式不同。
# GridSearchCV会遍历所有可能的参数组合，而RandomizedSearchCV则会随机选择一些参数组合进行训练。
# 当数据集较小时，我们可以使用GridSearchCV，但当数据集较大时，我们可以使用RandomizedSearchCV
random_search = RandomizedSearchCV(mlp, param_distributions=param_dist, cv=5)
random_search.fit(X_train, y_train)


C:\Users\lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=MLPClassifier(),
                   param_distributions={'activation': ['logistic', 'tanh',
                                                       'relu'],
                                        'hidden_layer_sizes': [(50,), (100,),
                                                               (150,)],
                                        'max_iter': [1000, 1500, 2000],
                                        'solver': ['sgd', 'adam']})

In [52]:
# 输出最佳超参数组合和对应的得分
print("Best parameters: {}".format(random_search.best_params_))
print("Best cross-validation score: {:.2f}".format(random_search.best_score_))

Best parameters: {'solver': 'adam', 'max_iter': 2000, 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
Best cross-validation score: 0.87


In [53]:
# 使用最佳超参数组合训练模型并预测测试集的label
mlp_best = MLPClassifier(**random_search.best_params_)
mlp_best.fit(X_train, y_train)
y_pred_mlp_best = mlp_best.predict(X_test)


In [54]:
# 计算MLP模型评估指标
accuracy_mlp_best = accuracy_score(y_test, y_pred_mlp_best)
precision_mlp_best = precision_score(y_test, y_pred_mlp_best)
recall_mlp_best = recall_score(y_test, y_pred_mlp_best)
f1_mlp_best = f1_score(y_test, y_pred_mlp_best)


In [55]:
print('\nMulti-layer Perceptron with best hyperparameters:')
print('Accuracy:', accuracy_mlp_best)
print('Precision:', precision_mlp_best)
print('Recall:', recall_mlp_best)
print('F1 Score:', f1_mlp_best)


Multi-layer Perceptron with best hyperparameters:
Accuracy: 0.9
Precision: 0.9230769230769231
Recall: 0.8571428571428571
F1 Score: 0.888888888888889


In [35]:
# don't test parameters

In [27]:
mlp = MLPClassifier(max_iter=10, hidden_layer_sizes=(10,), batch_size=128)

In [28]:
mlp.fit(X_train, y_train)

C:\Users\lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(batch_size=128, hidden_layer_sizes=(10,), max_iter=10)

In [29]:
# 预测测试集的label
y_pred_mlp = mlp.predict(X_test)

In [30]:
# 计算MLP模型评估指标
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

In [31]:
print('\nMulti-layer Perceptron:')
print('Accuracy:', accuracy_mlp)
print('Precision:', precision_mlp)
print('Recall:', recall_mlp)
print('F1 Score:', f1_mlp)


Multi-layer Perceptron:
Accuracy: 0.9346246973365617
Precision: 0.9678770949720671
Recall: 0.8985413290113452
F1 Score: 0.9319213313161876


In [34]:
# 将新的tweet_content转换为特征向量
# new_tweet_content = ['这是 一条 新的 正常 推文']

# depressed content
new_tweet_content = ['思 诺思 舒乐安定 代开 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']


# feature extraction
new_X = vectorizer.transform(new_tweet_content)

# 使用MLP模型进行预测
new_y_pred_mlp = mlp.predict(new_X)

print('MLP模型:', new_y_pred_mlp)

MLP模型: [1]
